<h1> InputDataSet Class and sktime example </h1>
First we import packages.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.classification.deep_learning.resnet import ResNetClassifier

import sys
sys.path.append('..')
# from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet

<h2> Define Features </h2>
Here we define the parameters for the InputDataSet object we will create. It includes the source of the data we want and a list of features we will restrict ourselves to.

In [ ]:
# We classify 5 characters on competitive stages

source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [ ]:
# We classify 5 characters on competitive stages

source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0'],
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['opposing_player'], # Can be 'general', 'player', 'opposing_player'
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [ ]:
# We classify 5 characters on competitive stages

source_data = ['ranked']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    'character_name': ['FOX'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    'character_name': ['FOX', 'JIGGLYPUFF'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0'],
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['opposing_player'], # Can be 'general', 'player', 'opposing_player'
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [ ]:
# We classify 5 characters on competitive stages

source_data = ['mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    'netplay_code': ['MANG#0'],
    'character_name': ['FALCO'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    'character_name': ['FOX', 'MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0'],
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['opposing_player'], # Can be 'general', 'player', 'opposing_player'
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}

<h2> Create Dataset Object </h2>
We create the dataset object and check the number of games for each label in our data and print the first several rows of the dataset to make sure it looks correct.

In [ ]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
dataset.dataset.head()

<h2> Call number_of_segments_per_game() </h2>
We set the length of the segments and the number of segments per label that we want in the dataset we are going to prepare. In the column 'Count' we see the number of games we have for each label (this is less than the value counts earlier because we discard games that are too short). In the column 'Shift' we see how much each segment will be shifted by. If the shift is less than the length of the segments, the data we prepare will contain overlaping segments.

In [ ]:
dataset.number_of_segments_per_game(10, 50000)

<h2> Split data and prepare it for the model </h2>

In [ ]:
X_train, X_test, y_train, y_test  = dataset.train_test_split_numpy(test_ratio = .30,  val = False)

# print()
# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)
# print(y_test.shape)

<h2> Train sktime model </h2>
Now that we have prepared our dataset, we train a model with sktime. After we train the model we calculate its score on the test data.

In [ ]:
# batch_size = 32
# epochs = 15

# model = CNNClassifier(verbose = 1, n_epochs=epochs, batch_size=batch_size, loss = 'categorical_crossentropy')
# model.fit(X_train, y_train)

# print('The test accuracy is: ', model.score(X_test, y_test))

# tf.compat.v1.reset_default_graph()

In [ ]:
batch_size = 32
epochs = 30

model = ResNetClassifier(verbose = 1, n_epochs=epochs, batch_size=batch_size, loss = 'categorical_crossentropy')
model.fit(X_train, y_train)

print('The test accuracy is: ', model.score(X_test, y_test))

tf.compat.v1.reset_default_graph()